# Homework 2

**Name:** -- Roberto José --

**e-mail:** -- roberto.jose0745@alumnos.udg.mx --

# Modules

In [6]:
# Load modules
import numpy as np
import math
import scipy.stats as stats
import plotly.graph_objects as go
import scipy.stats as stats
from scipy.stats import levy_stable

#Class Vec2d()

In [2]:
class Vec2d(object):
    """2D vector class with basic operations"""
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x, self.y = x_or_pair
        else:
            self.x, self.y = x_or_pair, y

    def __add__(self, other):
        return Vec2d(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vec2d(self.x - other.x, self.y - other.y)

    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    def rotated(self, angle):
        cos, sin = math.cos(angle), math.sin(angle)
        return Vec2d(self.x * cos - self.y * sin, self.x * sin + self.y * cos)


# Correlated Random Walk (CRW)
A Correlated Random Walk is a movement pattern where each step is influenced by the previous step's direction. Unlike regular random walks where each step is completely random, CRWs tend to keep going in similar directions.

This type of movement is commonly used to model animal foraging behavior, robot path planning, and diffusion processes in physics and biology.

1. Each step:
* Uses the previous direction as a guide
* Adds a small random change in angle
* Moves forward by a fixed distance

2. The math is straightforward:
* Start at point zero (0,0)
* Pick a direction
* For each step:
 * Slightly change direction
 * Move forward
 * Record new position

In [8]:
# Activity 1: Correlated Random Walk - 1 Trajectory
def correlated_random_walk(steps=100):
    pos = Vec2d(0, 0)
    trajectory = np.zeros((steps, 2))

    direction = Vec2d(1, 0)  # Right direction

    for i in range(steps):
        angle = stats.cauchy.rvs()  # Cauchy distribution - rotation
        direction = direction.rotated(angle)
        pos += direction
        trajectory[i] = [pos.x, pos.y]

    return trajectory

# Graph
trajectory = correlated_random_walk(200)

fig = go.Figure()
fig.add_trace(go.Scatter(x=trajectory[:, 0], y=trajectory[:, 1], mode='lines'))
fig.show()


#Lévy Distribution
A Lévy distribution is a type of probability curve that extends the normal bell curve to include more extreme outcomes. It's useful for modeling events with occasional big jumps, like stock prices or animal movements.
Key Points:

Has "fat tails" - extreme events happen more often than in normal distributions
Controlled by four main settings:

* $α$ (alpha): Controls tail thickness (smaller = more extreme values)
* $β$ (beta): Controls whether outcomes lean left or right
* $m$ (scale): Controls how spread out values are
* $μ$ (location): Controls the center point

Special Cases:
* When $α = 2$: Just a regular normal distribution
* When $α = 1$, $β = 0$: Cauchy distribution (very wide spread)
* When $α = 0.5$, $β = 1$: One-sided Lévy distribution

The math formula is complex, but the distribution is useful for modeling financial risks, physics problems, and animal search patterns where occasional big jumps occur.

In [11]:
# Activity 2: Lévy distribution - N different curves
# Parameters
alpha_values = [0.1, 1.0, 1.5, 1.9]
m, beta = 3.0, 0

# x-values
x = np.linspace(-10, 10, 500)

#Graph
fig = go.Figure()

for alpha in alpha_values:
    pdf = levy_stable.pdf(x, alpha, beta, loc=0, scale=m)
    fig.add_trace(go.Scatter(x=x, y=pdf, mode='lines', name=f'α = {alpha}'))

fig.show()


In [ ]:
# Plot the results